# Database

Content:
- Introduction to databases- Introduction to relational databases- Introduction to SQL + Python and Pandas

## Introduction to databases

So far, you&#39;ve probably worked with ** files ** on the course - they downloaded the file to your computer from somewhere, and processed it there, and then maybe they uploaded the result somewhere. However, this procedure is not enough if more people (or machines) need to work with this data at the same time and update it at the same time. This is solved by running ** database system ** (DBMS, [SŘBD] (https://cs.wikipedia.org/wiki/Syst%C3%A9m_%C5%99%C3%ADzen%C3%AD_b% C3% A1ze_dat)), which manages the data (files on disk containing specialized data structures) and you connect to this system via a computer network. Or maybe you are using a web application and it is connected to this database system (database).
E.g. The e-shop will work with a database in which information about the goods sold, division of goods into categories, orders, complaints, registered users is stored ... Such a database can be described as &quot;transactional&quot;.
You, as data analysts, engineers, etc., will probably want to connect to such a database and download some data from it for your future work. Or it may even be your job to create a new &quot;analytical&quot; database that contains data in another form, such as goods sold without labels, without photos, but with traffic and sales statistics.
In these materials we will work with the database [SQLite] (https://www.sqlite.org/). This is somewhat specific in that it is stored in a single file (with the extension `.sqlite` or` .db`), so it is not a stand-alone server that you would connect to, as is the case with most databases. But it&#39;s even easier to work with SQLite. SQLite also allows you to work with one database (one file) from several programs at the same time, unlike, for example, CSV or XLS files.

## Database Properties
We expect several features from the database (or database system):
- That it will store data :) No, seriously - the data must remain stored without loss even in the event of a power failure, when the disk runs out ...- Working with data &quot;from different angles&quot;. E.g. in the e-shop we want to list goods by category, other times by stock status, other times based on full-text search.- Support for users, roles, permissions and data access control, security.- Speed, power, throughput, latency- Scalability - both in size (gigabytes, terabytes, petabytes) and the number of operations performed per second- ACID
  - Atomicity:
  - Consistency:
  - Isolation:
  - Durability:
  
When you think about it, these are conflicting demands. Database system developers must choose a compromise. Sometimes the database can be configured to prefer isolation or throughput, for example (see [PostgreSQL Transaction Isolation] (https://www.postgresql.org/docs/9.5/transaction-iso.html)). No database is &quot;best&quot; and suitable for everything. As for the difference between a &quot;transactional&quot; and an &quot;analytical&quot; database, different systems and approaches are often appropriate for these areas.  

## Relational database
This is a typical, classic, most commonly used category of database systems. They have been used since the 1970s, there is a wide ecosystem around them with a lot of tools and also people who can work with them.
What [relational database] (https://en.wikipedia.org/wiki/Relational_database) contains:
- &quot;databases&quot; (one database system can manage several databases)- users- **tables**- **columns**- indices    - constraints
- foreign keys- triggery- views- procedures- sequence
The data is stored in the form of rows and columns in individual tables:
| title                 | year | director_name   | director_birthdate |
|:----------------------|:-----|:----------------|:-------------------|
| Lairs 1999 | Jan Hřebejk 1967-06-27 || Forrest Gump          | 1994 | Robert Zemeckis | 1951-05-14         |
| Back to the future 1985 | Robert Zemeckis | 1951-05-14 |
Each line is identified by its unique identifier, and duplicate data is * normalized *:
| movie_id | title                 | year | director_id |
|----------|:----------------------|:-----|:------------|
| 1 | Lairs 1999 | 1 ||        2 | Forrest Gump          | 1994 | 2           |
| 3 | Back to the future 1985 | 2 |
| director_id | name            | birthdate  |
|-------------|:----------------|------------|
| 1 | Jan Hřebejk 1967-06-27 ||           2 | Robert Zemeckis | 1951-05-14 |

The relationship between directors and films is 1: N. Every single director is associated with several films. Each film is linked to one director (or none if its director_id is not filled in).
What if we want to express the possibility that one film can have more than one director?
| movie_id | title                 | year | director_id |
|----------|:----------------------|:-----|:------------|
| 1 | Lairs 1999 | 1 ||        2 | Forrest Gump          | 1994 | 2           |
| 3 | Back to the future 1985 | 2 |
| director_id | name            | birthdate  |
|-------------|:----------------|------------|
| 1 | Jan Hřebejk 1967-06-27 ||           2 | Robert Zemeckis | 1951-05-14 |
|           3 | Joe Smith       | 1950-01-01 |

| movie_id | director_id |
|----------|-------------|
| 1        | 1           |
| 2        | 2           |
| 3        | 2           |
| 3        | 3           | 



## SQL

SQL is the language in which we communicate with a relational database. It can express various operations on tables - selection of columns, rows, relationships between multiple tables, aggregation of values from multiple rows ... (Doesn&#39;t that remind you of something? Pandas DataFrame? :))
Example:
```
SELECT title, year FROM movies;

UPDATE directors SET birthdate = "1950-01-02" WHERE director_id = 3;
```

Even software other than relational databases understands SQL. Often just analytical tools, data lakes, Amazon S3 ... Then there are a lot of databases and languages that were inspired by SQL - such as Cassandra and its CQL.

## Instalace a import SQLAlchemy

Each database from Python is used a little differently - the name of the module to be imported is different, the way data is used in SQL queries is slightly different, the form of data returned in the result is different ... We will therefore use the [SQLAlchemy] library (https: / /www.sqlalchemy.org/), which _abstracts_ these details and provides the same interface for all databases (which SQLAlchemy supports).
SQLAlchemy consists of [two parts] (https://docs.sqlalchemy.org/en/13/):
- Core - this is the aforementioned abstraction of basic database operations, this part we will use  - engine, connections, transactions
  - table metadata
  - SQL expression language
- [ORM] (https://docs.sqlalchemy.org/en/13/orm/tutorial.html) - allows to work with data in the database by mapping to objects- interesting especially when writing larger applications (web applications, etc.)

We will install the library [SQLAlchemy] (https://www.sqlalchemy.org/) via pip (don&#39;t forget to have venv activated in which you want to install it):
```
pip install sqlalchemy
```

Alternatively, the installation can be started directly from Jupyter:

In [1]:
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


We will import the SQLAlchemy library:

In [2]:
import sqlalchemy

Jako vždy, naimportujeme i [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html):

In [3]:
import pandas as pd

## Download movie database
We will use data from Míra Brabence to demonstrate working with the database.It is a database of type [SQLite] (https://cs.wikipedia.org/wiki/SQLite) - it consists of a single file, which contains all the data. This means that there is no large server to which you would connect by name and password, as in &quot;classic&quot; databases (PostgreSQL, Oracle, MS SQL ...).A database in the form of an SQLite file has the advantage that if you break something, you simply delete the file and make a new one :)
Download the database file with the library [Requests] (https://requests.readthedocs.io/en/master/), which you already have installed (instructions were in the first hour)and save it to a file in the working directory, ie where you started Jupyter Notebook.

In [4]:
from pathlib import Path
import requests

In [5]:
db_data = requests.get('https://github.com/messa/movie-db-2020/raw/master/movies.sqlite').content
Path('movies.sqlite').write_bytes(db_data)

1421312

## Open database, structure survey

In [6]:
engine = sqlalchemy.create_engine('sqlite:///movies.sqlite')

The `engine` object contains a connection to the database and through it we will perform operations with the given database.
Let&#39;s start by looking at the structure of that database - what tables it contains and what those tables contain. Each database has different tricks to find out this (sometimes the SQL statement `SHOW TABLES` works, other times` SELECT` from metadata is needed), fortunately SQLAlchemy will help us with that.

In [7]:
sqlalchemy.inspect(engine).get_table_names()

['actors', 'movie_to_actor', 'movies']

In [8]:
def explore (engine):    inspector = sqlalchemy.inspect(engine)
    for schema in inspector.get_schema_names():
print(&#39;Schedule:&#39;, schedule)        for table_name in inspector.get_table_names(schema=schema):
            print()
            print('  Table:', table_name)
            print()
            for column in inspector.get_columns(table_name, schema=schema):
                print('      Column:', column['name'].ljust(12), column['type'])

In [9]:
explore (engine)

Schema: main

  Table: actors

      Column: id           INTEGER
      Column: csfd_url     VARCHAR
      Column: name         VARCHAR
      Column: birth_date   DATE

  Table: movie_to_actor

      Column: id           INTEGER
      Column: movie_id     INTEGER
      Column: actor_id     INTEGER

  Table: movies

      Column: id           INTEGER
      Column: title        VARCHAR
      Column: csfd_url     VARCHAR
      Column: year         INTEGER
      Column: rating       NUMERIC


You can use a specialized program to explore the structure and contents of the database - [DB Browser] (https://sqlitebrowser.org/), [DBeaver] (https://dbeaver.io/), [MySQL Workbench] (https: // www.mysql.com/products/workbench/), ... there are many of them, typically specializing in a specific type of database, some programs are free and some paid. There are also web applications, the most famous is probably the Czech [Adminer] (https://www.adminer.org/). Here I wanted to show you that the same can be achieved from Python.

## SELECT

Here are some ways to &quot;pull&quot; data from a database into Python.

In [10]:
engine.execute('SELECT id, name, birth_date FROM actors')

The result of calling `engine.execute (...)` is an object that can represent the result of an SQL query. In the case of SELECT, the obtained rows can be passed through this object, in the case of INSERT, UPDATE, DELETE operations, this object contains, for example, the number of changed rows.
We will try to display individual lines. LIMIT is used to limit the number of records returned.

In [11]:
result = engine.execute('SELECT id, name, birth_date FROM actors LIMIT 5')
for row in result:
    print(row)

(1, 'Tim Robbins', '1958-10-16')
(2, 'Morgan Freeman', '1937-06-01')
(3, 'Bob Gunton', '1945-11-15')
(4, 'William Sadler', '1950-04-13')
(5, 'Clancy Brown', '1959-01-05')


Crawling `result` would seem to return n-tuples, but this is not the case.

In [12]:
result = engine.execute('SELECT id, name, birth_date FROM actors LIMIT 5')
for row in result:
    print(type(row))

<class 'sqlalchemy.engine.result.RowProxy'>
<class 'sqlalchemy.engine.result.RowProxy'>
<class 'sqlalchemy.engine.result.RowProxy'>
<class 'sqlalchemy.engine.result.RowProxy'>
<class 'sqlalchemy.engine.result.RowProxy'>


This is a SQLAlchemy RowProxy object that does a few interesting things:
- return the value of a given column in order- return the value of the given column by name- return dict with values

In [13]:
result = engine.execute('SELECT id, name, csfd_url FROM actors LIMIT 5')
for row in result:
    print(row[0], row[1])

1 Tim Robbins
2 Morgan Freeman
3 Bob Gunton
4 William Sadler
5 Clancy Brown


In [14]:
result = engine.execute('SELECT id, name, csfd_url FROM actors LIMIT 5')
for row in result:
    print(row['id'], row['name'])

1 Tim Robbins
2 Morgan Freeman
3 Bob Gunton
4 William Sadler
5 Clancy Brown


In [15]:
result = engine.execute('SELECT id, name, csfd_url FROM actors LIMIT 5')
for row in result:
    print(dict(row))

{'id': 1, 'name': 'Tim Robbins', 'csfd_url': 'https://www.csfd.cz/tvurce/103-tim-robbins/'}
{'id': 2, 'name': 'Morgan Freeman', 'csfd_url': 'https://www.csfd.cz/tvurce/92-morgan-freeman/'}
{'id': 3, 'name': 'Bob Gunton', 'csfd_url': 'https://www.csfd.cz/tvurce/202-bob-gunton/'}
{'id': 4, 'name': 'William Sadler', 'csfd_url': 'https://www.csfd.cz/tvurce/203-william-sadler/'}
{'id': 5, 'name': 'Clancy Brown', 'csfd_url': 'https://www.csfd.cz/tvurce/204-clancy-brown/'}


If we want to get values from all columns, we use an asterisk.

In [16]:
result = engine.execute('SELECT * FROM actors LIMIT 5')
for row in result:
    print(dict(row))

{'id': 1, 'csfd_url': 'https://www.csfd.cz/tvurce/103-tim-robbins/', 'name': 'Tim Robbins', 'birth_date': '1958-10-16'}
{'id': 2, 'csfd_url': 'https://www.csfd.cz/tvurce/92-morgan-freeman/', 'name': 'Morgan Freeman', 'birth_date': '1937-06-01'}
{'id': 3, 'csfd_url': 'https://www.csfd.cz/tvurce/202-bob-gunton/', 'name': 'Bob Gunton', 'birth_date': '1945-11-15'}
{'id': 4, 'csfd_url': 'https://www.csfd.cz/tvurce/203-william-sadler/', 'name': 'William Sadler', 'birth_date': '1950-04-13'}
{'id': 5, 'csfd_url': 'https://www.csfd.cz/tvurce/204-clancy-brown/', 'name': 'Clancy Brown', 'birth_date': '1959-01-05'}


Pandas DataFrame can also be created from the data:

In [17]:
pd.read_sql_query('SELECT id, name FROM actors LIMIT 5', engine)

,id,name
0,1,Tim Robbins
1,2,Morgan Freeman
2,3,Bob Gunton
3,4,William Sadler
4,5,Clancy Brown


This is a shortcut for loading the entire table:

In [18]:
pd.read_sql_table('actors', engine)

,id,csfd_url,name,birth_date
0,1,https://www.csfd.cz/tvurce/103-tim-robbins/,Tim Robbins,1958-10-16
1,2,https://www.csfd.cz/tvurce/92-morgan-freeman/,Morgan Freeman,1937-06-01
2,3,https://www.csfd.cz/tvurce/202-bob-gunton/,Bob Gunton,1945-11-15
3,4,https://www.csfd.cz/tvurce/203-william-sadler/,William Sadler,1950-04-13
4,5,https://www.csfd.cz/tvurce/204-clancy-brown/,Clancy Brown,1959-01-05
...,...,...,...,...
7522,7523,https://www.csfd.cz/tvurce/147104-jon-kenny/,Jon Kenny,NaT
7523,7524,https://www.csfd.cz/tvurce/147105-lisa-hannigan/,Lisa Hannigan,1981-02-12
7524,7525,https://www.csfd.cz/tvurce/162204-lucy-o-connell/,Lucy O'Connell,NaT
7525,7526,https://www.csfd.cz/tvurce/309228-paul-young/,Paul Young,NaT


## Survey data in a movie database
We will upload the data from the individual tables to the Pandas DataFrames, with which we will continue to work.
For clarity, we will write a sample of the content of those DataFrames here, so that we know what we are working with.

In [19]:
sqlalchemy.inspect(engine).get_table_names()

['actors', 'movie_to_actor', 'movies']

In [20]:
actors = pd.read_sql_table('actors', engine)
movies = pd.read_sql_table('movies', engine)
movie_to_actor = pd.read_sql_table('movie_to_actor', engine)

In [21]:
actors.head()

,id,csfd_url,name,birth_date
0,1,https://www.csfd.cz/tvurce/103-tim-robbins/,Tim Robbins,1958-10-16
1,2,https://www.csfd.cz/tvurce/92-morgan-freeman/,Morgan Freeman,1937-06-01
2,3,https://www.csfd.cz/tvurce/202-bob-gunton/,Bob Gunton,1945-11-15
3,4,https://www.csfd.cz/tvurce/203-william-sadler/,William Sadler,1950-04-13
4,5,https://www.csfd.cz/tvurce/204-clancy-brown/,Clancy Brown,1959-01-05


In [22]:
movies.head()

,id,title,csfd_url,year,rating
0,1,Vykoupení z věznice Shawshank,https://www.csfd.cz/film/2294-vykoupeni-z-vezn...,1994,95.3
1,2,Forrest Gump,https://www.csfd.cz/film/10135-forrest-gump/,1994,94.5
2,3,Zelená míle,https://www.csfd.cz/film/2292-zelena-mile/,1999,92.8
3,4,Přelet nad kukaččím hnízdem,https://www.csfd.cz/film/2982-prelet-nad-kukac...,1975,92.5
4,5,Sedm,https://www.csfd.cz/film/2671-sedm/,1995,92.4


In [23]:
movie_to_actor.head()

,id,movie_id,actor_id
0,1,1,1
1,2,1,2
2,3,1,3
3,4,1,4
4,5,1,5


## SELECT ... WHERE ... AND/OR

In [24]:
list(engine.execute('SELECT * FROM actors WHERE birth_date > "2000-01-01" AND birth_date < "2001-01-01"'))

[(4849, 'https://www.csfd.cz/tvurce/39309-medved-bart-2/', 'medvěd Bart 2', '2000-01-20'),
 (5703, 'https://www.csfd.cz/tvurce/283519-zaira-wasim/', 'Zaira Wasim', '2000-10-23'),
 (6290, 'https://www.csfd.cz/tvurce/35296-morgan-lily/', 'Morgan Lily', '2000-04-11')]

In [25]:
actors[(actors['birth_date'] > '2000-01-01') & (actors['birth_date'] < '2001-01-01')]

,id,csfd_url,name,birth_date
4848,4849,https://www.csfd.cz/tvurce/39309-medved-bart-2/,medvěd Bart 2,2000-01-20
5702,5703,https://www.csfd.cz/tvurce/283519-zaira-wasim/,Zaira Wasim,2000-10-23
6289,6290,https://www.csfd.cz/tvurce/35296-morgan-lily/,Morgan Lily,2000-04-11


## SELECT ... ORDER BY

In [26]:
list(engine.execute('SELECT * FROM actors WHERE birth_date > "2005-01-01" ORDER BY birth_date ASC'))

[(526, 'https://www.csfd.cz/tvurce/239127-noah-jupe/', 'Noah Jupe', '2005-02-25'),
 (7051, 'https://www.csfd.cz/tvurce/321955-rocco-lerro/', 'Rocco Lerro', '2005-08-21'),
 (4979, 'https://www.csfd.cz/tvurce/357987-ariana-greenblatt/', 'Ariana Greenblatt', '2007-08-27'),
 (1876, 'https://www.csfd.cz/tvurce/540779-hailey-hermida/', 'Hailey Hermida', '2007-09-13'),
 (1865, 'https://www.csfd.cz/tvurce/354371-pierce-pope/', 'Pierce Pope', '2008-06-02')]

In [27]:
actors[(actors['birth_date'] > '2005-01-01')].sort_values('birth_date')

,id,csfd_url,name,birth_date
525,526,https://www.csfd.cz/tvurce/239127-noah-jupe/,Noah Jupe,2005-02-25
7050,7051,https://www.csfd.cz/tvurce/321955-rocco-lerro/,Rocco Lerro,2005-08-21
4978,4979,https://www.csfd.cz/tvurce/357987-ariana-green...,Ariana Greenblatt,2007-08-27
1875,1876,https://www.csfd.cz/tvurce/540779-hailey-hermida/,Hailey Hermida,2007-09-13
1864,1865,https://www.csfd.cz/tvurce/354371-pierce-pope/,Pierce Pope,2008-06-02


## SELECT ... GROUP BY

In [28]:
list(engine.execute('SELECT year, COUNT(*) FROM movies GROUP BY year HAVING COUNT(*) > 5'))

[(1957, 6),
 (1968, 6),
 (1973, 6),
 (1980, 6),
 (1984, 6),
 (1986, 7),
 (1990, 7),
 (1991, 6),
 (1993, 11),
 (1995, 7),
 (1997, 8),
 (1998, 6),
 (1999, 9),
 (2000, 6),
 (2001, 8),
 (2002, 6),
 (2003, 6),
 (2004, 9),
 (2008, 6)]

## Task: which actor acted in the largest number of films
In the `movie_to_actor` table, find the id of the actor with the most movie ids in that table.
Method:
1. Use GROUP BY to always write down actor_id and the number of lines with this actor_id2. Use ORDER BY and LIMIT to find the actor_id with the highest number3. Bonus: find out the name of this actor

## JOIN

TODO: here will be a sample JOIN in SQL vs. merge in the DataFrame
<div><img src="https://naucse.python.cz/2020/pydata-praha-jaro/pydata/pandas_correlations/static/joins.svg"></div>

## CREATE TABLE

## INSERT

## UPDATE

## DELETE

## Conclusion

## Links
https://naucse.python.cz/2019/brno-podzim-2019-ut/beginners/database/
